# Stock Simulator

In [8]:
# Imports
import numpy as np
import random
import matplotlib.pyplot as plt
import time
import pandas as pd
from ipywidgets import widgets
from IPython.display import display, clear_output, HTML

In [15]:
# IPyWidgets GUI
initial_amount_label = widgets.Label("Initial Amount ($):")
initial_amount_text = widgets.Text()

goal_amount_label = widgets.Label("Goal Amount ($):")
goal_amount_text = widgets.Text()

profit_loss_range_label = widgets.Label("P&L Range (%):")
profit_loss_range_slider = widgets.IntRangeSlider(min = -100, max = 100)

perc_success_label = widgets.Label("Success Rate (%):")
perc_success_slider = widgets.IntSlider(value = 50, min = 0, max = 100)

run_button = widgets.Button(description = "Start")
mc_radio = widgets.Checkbox(False, description = "MC")

validate_out = widgets.Output()
graph_out = widgets.Output()
right_out = widgets.Output()

progress = widgets.FloatProgress(description = "Loading...", value = 0.0, min = 0.0, max = 1.0)

def stock_simulator(run_button):
    """
    This method simulates a stock portfolio over time
    """
    validate_out.clear_output()
    graph_out.clear_output()
    right_out.clear_output()   
    
    # Input validation / Loading...
    with validate_out:
        if (initial_amount_text.value.replace('.', '', 1).isdigit() == False or goal_amount_text.value.replace('.', '', 1).isdigit() == False):
            print("Invalid input. Initial and/or goal amount cannot be nonnumeric.")
            return
        if (float(initial_amount_text.value) < 0 or float(goal_amount_text.value) < 0):
            print("Invalid input. Initial and/or goal amount cannot be negative.")
            return
        if (float(goal_amount_text.value) < float(initial_amount_text.value)):
            print("Invalid input. Goal amount cannot be < initial amount.")
            return
        if (profit_loss_range_slider.value == (0, 0)):
            print("Invalid input. P&L range cannot be 0%-0%.")
            return
    
    days_finished = []
    successful_trials = 0
    
    n = 1000 if mc_radio.value == True else 1
    
    for i in range(n):
        
        # Initialization
        days_lst = []
        amount_lst = []
        return_lst = []
        days = 0
        
        # Cleans input data
        amount = float(initial_amount_text.value)
        goal = float(goal_amount_text.value)
        profit_loss_range = [i * 0.01 for i in profit_loss_range_slider.value]
        if profit_loss_range[0] >= 0 and profit_loss_range[1] >= 0:
            perc_success_slider.value = 100
        elif profit_loss_range[0] <= -0.01 and profit_loss_range[1] <= -0.01:
            perc_success_slider.value = 0      
        perc_success = perc_success_slider.value * 0.01

        # Initial day
        days_lst.append(days)
        amount_lst.append(amount)
        return_lst.append("+$0.00 (+0.00%)")

        while amount < goal:

            # Ends program when you lost your whole port :(
            if int(amount) <= 0:
                break
            
            decision = np.random.choice([True, False], p = [perc_success, 1 - perc_success])
                
            # Gain
            if decision:
                lower = max(0, profit_loss_range[0])
                upper = profit_loss_range[1]
                gain_perc = random.uniform(lower, upper)
                gain = amount * gain_perc
                amount += gain
                return_lst.append("+${:,.2f} ".format(gain) + "(+{:,.2f}%)".format(gain_perc * 100))

            # Loss
            else:
                lower = profit_loss_range[0]
                upper = min(-0.01, profit_loss_range[1])
                loss_perc = random.uniform(lower, upper) # upper biased
                loss = amount * loss_perc
                amount += loss
                return_lst.append("-${:,.2f} ".format(abs(loss)) + "({:,.2f}%)".format(loss_perc * 100))

            # Adds data to lists
            days += 1
            days_lst.append(days)
            amount_lst.append(amount)
            
        days_finished.append(days)
        successful_trials += 1
                
        # Line graph
        with graph_out:    
            if i == 0:
                display(progress)
                plt.figure(figsize = (15, 6))
                plt.title("Time vs. Portfolio Value (n={})".format(n))
                plt.xlabel("Time (Day)")
                plt.ylabel("Portfolio Value ($)")
                
            plt.plot(days_lst, amount_lst)
            
            # Updates progress bar
            time.sleep(0.2)
            progress.value = float(i + 1) / n
            
            if i == n - 1:
                graph_out.clear_output()
                plt.show()
                
                if mc_radio.value == True:
                    
                    with right_out:
                        print("You were {:.0%} successful.".format(successful_trials / n))
                        print("\nSummary Stats (for your successes):")
                        days_finished_series = pd.Series(days_finished)
                        print(days_finished_series.describe())
            
            if mc_radio.value == False:
                # Prints when goal is reached
                if amount >= goal:
                    print("It will take you " + f"{days:,}" + " days to reach your goal!")
                # Prints when you lose all your money
                else:
                    print("You lost all your money :(")
        
        if mc_radio.value == False:
            
            # DataFrame
            with right_out:
                df = pd.DataFrame().assign(Day = days_lst, Amount = amount_lst).set_index("Day")
                df["Amount"] = df["Amount"].apply(lambda val : "${:,.2f}".format(val))
                df = df.assign(Return = return_lst)
                # Displays DataFrame with scroll wheel
                pd.set_option("display.max_rows", None)
                display(HTML("<div style='height: 400px; overflow: auto; width: fit-content'>" + df.style.render() + "</div>"))
        
    validate_out.clear_output()
    
run_button.on_click(stock_simulator)

run_button_mc_radio = widgets.HBox([run_button, mc_radio])

menu = widgets.VBox([initial_amount_label, initial_amount_text, \
                     goal_amount_label, goal_amount_text, \
                     profit_loss_range_label, profit_loss_range_slider, \
                     perc_success_label, perc_success_slider, \
                     run_button_mc_radio, \
                     validate_out])

page = widgets.HBox([menu, graph_out, right_out])
display(page)